In [0]:
from pyspark.sql.functions import current_timestamp, lit, concat, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
name_schema = StructType(fields= [
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
persons_schema = StructType(fields= [
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
])

In [0]:
person_df = spark.read.schema(persons_schema) \
                    .json("/mnt/storymovie/bronze/person.json")

In [0]:
persons_columnas_df = person_df.withColumnRenamed('personId', 'person_id') \
                                .withColumn('ingestion_date', current_timestamp()) \
                                .withColumn('env', lit('python')) \
                                .withColumn('name', concat(
                                                        col("personName.forename"), 
                                                        lit(" "), 
                                                        col("personName.surname"))
                                                            )

In [0]:
person_final_df = persons_columnas_df.drop("personName")

In [0]:
display(person_final_df)

In [0]:
person_final_df.write.mode("overwrite").parquet("/mnt/storymovie/silver/persons")

In [0]:
%fs
ls /mnt/storymovie/silver/